In [ ]:
import torch
import re
import pandas as pd
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
df_rad =pd.read_csv('/Users/deepureddy/Desktop/REDI/radiology_reports_birads_score_STUDENTS_PRO.csv')
df_rad = df_rad.apply(lambda x: x.astype(str).str.lower())
df_rad["BIRADS"] = df_rad["BIRADS"].replace("i",1)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("ii",2)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("iii",3)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("iv",4)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("v",5)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("vi",6)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("vii",7)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("viii",8)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("ix",9)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("x",10)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("een",1)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("twee",2)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("drie",3)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("vier",4)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("vijf",5)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("zes",6)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("zeven",7)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("acht",8)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("negen",9)
df_rad["BIRADS"] = df_rad["BIRADS"].replace("tien",10)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('iva', 4)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('ivb', 4)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('ivc', 4)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('ivd', 4)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('4a', 4)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('4b', 4)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('4c', 4)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('4d', 4)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('0', 0)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('1', 1)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('2', 2)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('3', 3)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('4', 4)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('5', 5)
df_rad['BIRADS'] = df_rad['BIRADS'].replace('6', 6)
df_rad_1 = df_rad
import pandas as pd
import numpy as np
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('dutch')
stop.remove('geen')
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–","\n",'ii','iii',"\d","Verslag:", "medische", "gegevens"]
for char in spec_chars:
    df_rad_1['Report_MG'] = df_rad_1['Report_MG'].str.replace(char, ' ')
    df_rad_1['Report_US'] = df_rad_1['Report_US'].str.replace(char, ' ')
    df_rad_1['Report_MRI'] = df_rad_1['Report_MRI'].str.replace(char, ' ')
df_rad_1['Report_MG'] = df_rad_1['Report_MG'].str.split("Conclusie:",expand=True)
df_ploted = df_rad_1['BIRADS'].value_counts()
df_ploted.plot.bar(x="test", y="gest", rot=100, title="BIRADS Data distrubtion")
df_rad_1.drop(df_rad_1.index[df_rad_1['BIRADS'] == 'nan'], inplace = True)
df_rad_1.drop(df_rad_1.index[df_rad_1['BIRADS'] == '7'], inplace = True)
df_rad_1.drop(df_rad_1.index[df_rad_1['BIRADS'] == '8'], inplace = True)
df_rad_1.drop(df_rad_1.index[df_rad_1['BIRADS'] == '9'], inplace = True)
df_ploted = df_rad_1['BIRADS'].value_counts()
df_ploted.plot.bar(x="test", y="gest", rot=100, title="BIRADS Data distrubtion")
df_training = df_rad_1[['Id','Report_MG','BIRADS']]
df_training['cleaned_text'] = df_training['Report_MG'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_training.drop(df_training.index[df_training['Report_MG'] == 'nan'], inplace = True)
new_ploted = df_training['BIRADS'].value_counts()
new_ploted.plot.bar(title="BIRADS Data distrubtion")
df_training['cleaned_text'] = df_training['Report_MG'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_training.head()
df_main_data = df_training[['cleaned_text','BIRADS']]

In [ ]:
df_main_data

In [ ]:
import numpy as np
seq_len = 512
num_samples = len(df_main_data)
Xids =np.zeros((num_samples,seq_len))
Xmask =np.zeros((num_samples,seq_len))
Xids.shape

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')
for i,cleaned_text in enumerate(df_main_data['cleaned_text']):
    tokens = tokenizer.encode_plus(cleaned_text, max_length=seq_len, truncation = True,
                                 padding ='max_length',add_special_tokens = True,return_tensors = 'tf')
    Xids[i, :] = tokens['input_ids']
    Xmask[i, :] = tokens['attention_mask']

In [ ]:
Xids

In [ ]:
Xmask

In [ ]:
df_main_data

In [ ]:
arr = pd.to_numeric(df_main_data['BIRADS'].values)
arr

In [ ]:
labels = np.zeros((num_samples, arr.max()+1))
labels.shape

In [ ]:
#df_main_data['BIRADS'] = df_main_data['BIRADS'].astype(int)
df_main_data['BIRADS'] = pd.to_numeric(df_main_data['BIRADS'])

df_main_data.dtypes

In [ ]:
labels[np.arange(num_samples), arr] = 1

In [ ]:
labels

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((Xids,Xmask,labels))
dataset.take(1)

In [ ]:
labels[0, :].shape

In [ ]:
def map_func(input_ids, masks,labels):
    return {'input_ids' :input_ids, 'attention_mask' : masks}, labels

In [ ]:
dataset = dataset.map(map_func)

In [ ]:
dataset.take(1)

In [ ]:
batch_size = 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder = True)
dataset.take(1)

In [ ]:
split = 0.8
size = int((num_samples/batch_size)*split)

In [ ]:
int((num_samples/batch_size)*split)

In [ ]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)
del dataset

In [ ]:
from transformers import TFAutoModel
bert = TFAutoModel.from_pretrained('GroNLP/bert-base-dutch-cased')
bert.summary()

In [ ]:
input_ids = tf.keras.layers.Input(shape=(seq_len,), name = 'input_ids',dtype='int32')
mask = tf.keras.layers.Input(shape=(seq_len,), name = 'attention_mask',dtype='int32')
embeddings = bert.bert(input_ids,mask)[1]
x = tf.keras.layers.Dense(1024, activation = 'relu')(embeddings)
y = tf.keras.layers.Dense(arr.max()+1, activation = 'softmax',name = 'outputs')(x)

In [ ]:
model = tf.keras.Model(inputs =[input_ids,mask], outputs = y)
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.1, decay = 0.1)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
model.compile(optimizer=optimizer,loss =loss, metrics = [acc])

In [ ]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs =1 
)

In [ ]:
model.save('sentiment_model')

In [ ]:
import tensorflow as tf

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')
def pred_data(text):
    tokens = tokenizer.encode_plus(text,max_length = 512, truncation = True, padding = 'max_length',
                                  add_special_tokens = True,return_token_type_id = False, return_tensors = 'tf')
    return {
        'input_ids' : tf.cast(tokens['input_ids'],tf.float64),
        'attention_mask' : tf.cast(tokens['attention_mask'],tf.float64)

    }

In [ ]:
test = pred_data('aanhoudende pijnklachten linkerborst geen palpabele afwijkingen verslag mammografie volgens protocol ter vergelijk analoge onderzoek d d goed beoordeelbaar mammogram deels geïnvolueerd fibroglandulair weefsel acr classificatie geringe asymmetrie verdeling klierweefsel waarbij rechts lateraal druk gebied aanvullende detailopname gebied geen verdachte massa s densiteiten anderszins tevens geen verdachte massa s densiteiten geen architectuurverstoringen geen clusters microcalcificaties normaal beeld cutis subcutis alsmede mamilla beiderzijds axillae geen aanwijzingen pathologie conclusie normaal mammogram geen aanwijzingen maligniteit birads i')

In [ ]:
prob = model.predict(test)
prob[0]

In [ ]:
import numpy as np
Y_pred = np.argmax(prob[0])

In [ ]:
cf_matrix = confusion_matrix(y, y_pred,labels)
print(cf_matrix)